In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/VT_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3094947107,VT,"193 TILLEY DR, SOUTH BURLINGTON, VT",44.449194,-73.149491,5.000700e+14
1,5465908809,VT,"1 Anna Marsh Lane, Brattleboro, VT",42.858816,-72.562629,5.002597e+14
2,5465897006,VT,"239 PLEASANT ST, BETHEL, VT",43.831905,-72.642580,5.002797e+14
3,3949557104,VT,"1 COLLEGE DR, BENNINGTON, VT",42.921399,-73.236727,5.000397e+14
4,5449437008,VT,"210 PIERCE RD, SAINT JOHNSBURY, VT",44.480466,-72.011581,5.000596e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3094947107,VT,"193 TILLEY DR, SOUTH BURLINGTON, VT",44.449194,-73.149491,5.000700e+14
1,5465908809,VT,"1 Anna Marsh Lane, Brattleboro, VT",42.858816,-72.562629,5.002597e+14
2,5465897006,VT,"239 PLEASANT ST, BETHEL, VT",43.831905,-72.642580,5.002797e+14
3,3949557104,VT,"1 COLLEGE DR, BENNINGTON, VT",42.921399,-73.236727,5.000397e+14
4,5449437008,VT,"210 PIERCE RD, SAINT JOHNSBURY, VT",44.480466,-72.011581,5.000596e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
725,4395107002,VT,"4441 US ROUTE 2, DERBY, VT",48.313811,-114.244046,NaN
790,4988897305,VT,"1100 U.S. Route 2, MONTPELIER, VT",44.361078,-71.343354,NaN
2069,5687147010,VT,"59 US Route 4 East, WOODSTOCK, VT",43.264344,-73.582204,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('50')] #VT
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/3 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

3


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_15541/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
725,4395107002,VT,"4441 US ROUTE 2, DERBY, VT",48.313811,-114.244046,[]
790,4988897305,VT,"1100 U.S. Route 2, MONTPELIER, VT",44.361078,-71.343354,[]
2069,5687147010,VT,"59 US Route 4 East, WOODSTOCK, VT",43.264344,-73.582204,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_15541/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
725,4395107002,VT,"4441 US ROUTE 2, DERBY, VT",48.313811,-114.244046,NaN
790,4988897305,VT,"1100 U.S. Route 2, MONTPELIER, VT",44.361078,-71.343354,NaN
2069,5687147010,VT,"59 US Route 4 East, WOODSTOCK, VT",43.264344,-73.582204,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
725,4395107002,VT,"4441 US ROUTE 2, DERBY, VT",48.313811,-114.244046,NaN
790,4988897305,VT,"1100 U.S. Route 2, MONTPELIER, VT",44.361078,-71.343354,NaN
2069,5687147010,VT,"59 US Route 4 East, WOODSTOCK, VT",43.264344,-73.582204,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
725,4395107002,NaN
790,4988897305,NaN
2069,5687147010,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/VT_fips_scraped.csv") 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,916811,3094947107,VT,"193 TILLEY DR, SOUTH BURLINGTON, VT",44.449194,-73.149491,5.000700e+14
1,916812,5465908809,VT,"1 Anna Marsh Lane, Brattleboro, VT",42.858816,-72.562629,5.002597e+14
2,916813,5465897006,VT,"239 PLEASANT ST, BETHEL, VT",43.831905,-72.642580,5.002797e+14
3,916814,3949557104,VT,"1 COLLEGE DR, BENNINGTON, VT",42.921399,-73.236727,5.000397e+14
4,916815,5449437008,VT,"210 PIERCE RD, SAINT JOHNSBURY, VT",44.480466,-72.011581,5.000596e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,916811,3094947107,VT,"193 TILLEY DR, SOUTH BURLINGTON, VT",44.449194,-73.149491,5.000700e+14,NaN
1,916812,5465908809,VT,"1 Anna Marsh Lane, Brattleboro, VT",42.858816,-72.562629,5.002597e+14,NaN
2,916813,5465897006,VT,"239 PLEASANT ST, BETHEL, VT",43.831905,-72.642580,5.002797e+14,NaN
3,916814,3949557104,VT,"1 COLLEGE DR, BENNINGTON, VT",42.921399,-73.236727,5.000397e+14,NaN
4,916815,5449437008,VT,"210 PIERCE RD, SAINT JOHNSBURY, VT",44.480466,-72.011581,5.000596e+14,NaN
...,...,...,...,...,...,...,...,...
2260,919071,8117198404,VT,"359 South St, New Haven, VT",44.118951,-73.152154,5.000196e+14,NaN
2261,919072,8325598705,VT,"39 S Main St, White River Junction, VT",43.648022,-72.318954,5.002797e+14,NaN
2262,919073,8759258607,VT,"1100 Hinesburg Rd Ste 201, South Burlington, VT",44.440169,-73.157060,5.000700e+14,NaN
2263,919074,8777268309,VT,"56 College St Ste 201, Montpelier, VT",44.256352,-72.566071,5.002395e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,916811,3094947107,VT,"193 TILLEY DR, SOUTH BURLINGTON, VT",44.449194,-73.149491,5.000700e+14,NaN,5.000700e+14
1,916812,5465908809,VT,"1 Anna Marsh Lane, Brattleboro, VT",42.858816,-72.562629,5.002597e+14,NaN,5.002597e+14
2,916813,5465897006,VT,"239 PLEASANT ST, BETHEL, VT",43.831905,-72.642580,5.002797e+14,NaN,5.002797e+14
3,916814,3949557104,VT,"1 COLLEGE DR, BENNINGTON, VT",42.921399,-73.236727,5.000397e+14,NaN,5.000397e+14
4,916815,5449437008,VT,"210 PIERCE RD, SAINT JOHNSBURY, VT",44.480466,-72.011581,5.000596e+14,NaN,5.000596e+14


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3094947107,VT,"193 TILLEY DR, SOUTH BURLINGTON, VT",44.449194,-73.149491,5.000700e+14
1,5465908809,VT,"1 Anna Marsh Lane, Brattleboro, VT",42.858816,-72.562629,5.002597e+14
2,5465897006,VT,"239 PLEASANT ST, BETHEL, VT",43.831905,-72.642580,5.002797e+14
3,3949557104,VT,"1 COLLEGE DR, BENNINGTON, VT",42.921399,-73.236727,5.000397e+14
4,5449437008,VT,"210 PIERCE RD, SAINT JOHNSBURY, VT",44.480466,-72.011581,5.000596e+14
...,...,...,...,...,...,...
2260,8117198404,VT,"359 South St, New Haven, VT",44.118951,-73.152154,5.000196e+14
2261,8325598705,VT,"39 S Main St, White River Junction, VT",43.648022,-72.318954,5.002797e+14
2262,8759258607,VT,"1100 Hinesburg Rd Ste 201, South Burlington, VT",44.440169,-73.157060,5.000700e+14
2263,8777268309,VT,"56 College St Ste 201, Montpelier, VT",44.256352,-72.566071,5.002395e+14


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
725,4395107002,VT,"4441 US ROUTE 2, DERBY, VT",48.313811,-114.244046,NaN
790,4988897305,VT,"1100 U.S. Route 2, MONTPELIER, VT",44.361078,-71.343354,NaN
2069,5687147010,VT,"59 US Route 4 East, WOODSTOCK, VT",43.264344,-73.582204,NaN


Convert to File

In [19]:
fips_VT = fips_merge_drop

In [20]:
fips_VT.to_csv("../../../data/state_data/geo/geo_fips/23/VT_fips_scraped.csv") 